In [1]:
import numpy as np
import torch as th
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
from torch import nn, optim
import torchvision


In [ ]:
plt.style.use('seaborn-poster')

In [ ]:
trainsforms = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

In [ ]:
def dataset(file):
    current_datafile = open(file, 'r')
    current_dataset = current_datafile.readlines()
    current_datafile.close()

    datasets = np.zeros([len(current_dataset), 1025])
    indices = np.linspace(0, len(current_dataset) - 1, len(current_dataset), dtype = np.int64)
    np.random.shuffle(indices)
    k = 0
    for i in indices:
        
        example = current_dataset[i].split(',')
        labels = np.asarray(example[0], dtype = np.float32)
        images = np.asarray(example[1:], dtype=np.float32) / 255   
        example = np.hstack([labels, images])
        
        datasets[k] = example
        k = k + 1
    return datasets

In [ ]:
train_data = dataset('train.csv')
test_data = dataset('test.csv')
trainloader = th.utils.data.DataLoader(train_data, shuffle = False)
testloader = th.utils.data.DataLoader(test_data, shuffle = True)

In [ ]:
class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(1024, 512)
#         self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 10)
        
        self.drop = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
#         x = self.drop(F.relu(self.fc2(x)))
        x = self.drop(F.relu(self.fc3(x)))
        x = F.softmax(self.fc4(x))
        
        return x
    
model = FNN()

    


In [ ]:
# Check if model working fine
example = next(iter(trainloader))
label = example[0][0]
example = example[0][1:].float()
# example = example.reshape(1, 1, 32, 32)

model(example)
# print(model.fc1.weight.data, model.fc3.weight.data)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)
criterion = nn.CrossEntropyLoss()


In [ ]:
epochs = 2
for epoch in range(epochs):
    running_loss = 0
    for example in trainloader:
    #     example = example.reshape(1025).float()
        label = th.Tensor([example[0][0].float()]).long()
        image = example[0][1:].float()
                
        print("1: -->", model.fc1.weight.data, model.fc3.weight.data)
        
        optimizer.zero_grad()
        
        logps = model(image).reshape(1, 10)
        loss = criterion(logps, label)
#         print(logps, label)
        
        loss.backward()
        optimizer.step()
        
        print("2: -->", model.fc1.weight.data, model.fc3.weight.data)
        running_loss += loss.item()

    else:
        accuracy = 0
        
        print(f"Loss: {running_loss / len(trainloader)}")

In [ ]:
example = next(iter(testloader))

label = example[0][0]
image = th.reshape(example[0][1:].float(), (1, 1024))
# image = image.view(image.shape[0], -1)
prediction = model(image)

pred_label = th.argmax(prediction)

title = f"Actual: {int(label)} Prediction: {pred_label}"

plt.imshow(image.reshape(32, 32))
plt.title(title)
plt.show()